# Parameterizing Notebooks

The [Databricks Utilities module](https://docs.databricks.com/dev-tools/databricks-utils.html) includes a number of methods to make notebooks more extensible and easier to take to production. This notebook is designed to be scheduled as a job, but can also be run interactively.

### Learning Objectives
- Pass parameters to notebooks using widgets
- Return values from notebooks using exit value

In [2]:
%run "./Includes/Classroom-Setup"

## Widgets

The `widgets` submodule includes a number of methods to allow interactive variables to be set while working with notebooks in the workspace with an interactive cluster. To learn more about this functionality, refer to the [Databricks documentation](https://docs.databricks.com/notebooks/widgets.html#widgets).

This notebook will focus on only two of these methods, emphasizing their utility when running a notebook as a job:
1. `dbutils.widgets.text` accepts a parameter name and a default value. This is the method through which external values can be passed into scheduled notebooks.
1. `dbutils.widgets.get` accepts a parameter name and retrieves the associated value from the widget with that parameter name.

-sandbox
In the cell below, a text widget is created with the default value `"notebook"`. This widget expects values to be passed as strings.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> If you run this cell in an interactive notebook, you will see the widget populated with the default value at the top of the notebook. This can be manually manipulated.

In [5]:
%scala
dbutils.widgets.text("ranBy", "notebook")

-sandbox
The cell below retrieves the value currently associated with the widget and assigns it to a variable. Remember that this value will be passed as a string--be sure to cast it to the correct type if you wish to pass numeric values or use JSON to pass multiple fields.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> If no parameter is passed to the notebook when scheduling, the default value will be used.

In [7]:
%scala
val ranBy = dbutils.widgets.get("ranBy")

ranBy: String = notebook

Taken together, `dbutils.widgets.text` allows the passing of external values and `dbutils.widgets.get` allows those values to be referenced.

## Parameterized Logic
The following code block writes a simple file that records the time the notebook was run and the value associated with the `"ranBy"` parameter/widget. The final line displays the full content of this file from all previous executions by the present user.

In [10]:
%scala
import org.apache.spark.sql.functions.{lit, unix_timestamp}
import org.apache.spark.sql.types.TimestampType

val tags = com.databricks.logging.AttributionContext.current.tags
val username = tags.getOrElse(com.databricks.logging.BaseTagDefinitions.TAG_USER, java.util.UUID.randomUUID.toString.replace("-", ""))
val path = username+"/runLog"

spark
  .range(1)
  .select(unix_timestamp.alias("runtime").cast(TimestampType), lit(ranBy).alias("ranBy"))
  .write
  .mode("APPEND")
  .parquet(path)

display(spark.read.parquet(path))

at com.databricks.common.path.AbstractPath$.fromHadoopPath(AbstractPath.scala:114)
	at com.databricks.backend.daemon.data.client.DBFSV2.resolve(DatabricksFileSystemV2.scala:80)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$2(DatabricksFileSystemV2.scala:760)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:108)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$1(DatabricksFileSystemV2.scala:759)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:430)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:453)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:268)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:453)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:411)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:337)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:453)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.getFileStatus(DatabricksFileSystemV2.scala:759)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.getFileStatus(DatabricksFileSystem.scala:201)
	at org.apache.hadoop.fs.FileSystem.isDirectory(FileSystem.java:1439)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:47)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:387)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:366)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:355)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:355)
	at org.apache.spark.sql.DataFrameReader.parquet(DataFrameReader.scala:839)
	at org.apache.spark.sql.DataFrameReader.parquet(DataFrameReader.scala:817)
	at linef33a04f525764c139ae2047cfb6ca68737.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-1947196994887888:15)
	at linef33a04f525764c139ae2047cfb6ca68737.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-1947196994887888:70)
	at linef33a04f525764c139ae2047cfb6ca68737.$read$$iw$$iw$$iw$$iw.<init>(command-1947196994887888:72)
	at linef33a04f525764c139ae2047cfb6ca68737.$read$$iw$$iw$$iw.<init>(command-1947196994887888:74)
	at linef33a04f525764c139ae2047cfb6ca68737.$read$$iw$$iw.<init>(command-1947196994887888:76)
	at linef33a04f525764c139ae2047cfb6ca68737.$read$$iw.<init>(command-1947196994887888:78)
	at linef33a04f525764c139ae2047cfb6ca68737.$read.<init>(command-1947196994887888:80)
	at linef33a04f525764c139ae2047cfb6ca68737.$read$.<init>(command-1947196994887888:84)
	at linef33a04f525764c139ae2047cfb6ca68737.$read$.<clinit>(command-1947196994887888)
	at linef33a04f525764c139ae2047cfb6ca68737.$eval$.$print$lzycompute(<notebook>:7)
	at linef33a04f525764c139ae2047cfb6ca68737.$eval$.$print(<notebook>:6)
	at linef33a04f525764c139ae2047cfb6ca68737.$eval.$print(<notebook>)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at scala.tools.nsc.interpreter.IMai

## Exit Value
The `notebook` submodule contains only two methods. [Documentation here](https://docs.databricks.com/notebooks/notebook-workflows.html#notebook-workflows).
1. `dbutils.notebook.run` allows you to call another notebook using a relative path.
1. `dbutils.notebook.exit` allows you to return an exit value that can be captured and referenced by integrated scheduling services and APIs. While running in interactive mode, this is essentially a no-op as this value does not go anywhere.

In the cell below, the value associated with the variable `path` is returned as the exit value.

In [13]:
%scala
dbutils.notebook.exit(path)
